# Cython

* Cython is one of python's *dialects* to bridge between C and python.

* Following code block is an [example](https://stackoverflow.com/questions/35656604/running-cython-in-jupyter-ipython) of the `cython` code.

In [ ]:
%load_ext Cython
# make cython feature avaiable in ipython 


In [ ]:
%%cython --annotate
# with command above, ipython would regard this cell as cython
# "%%" must be on the first line of the cell
# --annotate option would present c code from the cython code

# declare a cython function
def geo_prog_cython(double alpha, int n):
    # this function has two arguments
    # alpha is double precision float
    # n is an integer (probably 32bit)
    
    # local variables for this function 
    # with types and initial values
    cdef double current = 1.0
    cdef double sum = current
    cdef int i
    
    # accumulation loop
    for i in range(n):
        # current term of a geometric sequence
        current = current * alpha
        # accumulation
        sum += current

    # sum of the geometric sequence    
    return sum



In [ ]:
%%time
# the command above would measure time of this cell

geo_prog_cython(0.5, 5)



* To compare the computation time, let's prepare a python version.

In [ ]:
def geo_prog_python(alpha, n):
    current = 1.0
    sum = current
    
    for i in range(n):
        current = current * alpha
        sum += current
    return sum



In [ ]:
%%time

geo_prog_python(0.5, 5)



## Visualizing

* `matplotlib` can visualize results from `cython` functions.

* Following code presents Euler's method simulation.

In [ ]:
%load_ext Cython



In [ ]:
%%cython --annotate

def diff_eq(xi):
    """
    Differential equation to solve
    x' + x = 0
    x' = - x
    """

    # dx/dt + x = 0
    # return value == result of this function
    return -xi


def euler_cython():
    # a simplified version of Euler's method in cython

    # initial values
    # start time
    cdef double ti = 0.0
    # end time
    cdef double te = 10.0
    # time interval
    cdef double delta_t = 1e-3
    # inital value
    cdef double x0 = 1.0
    
    # time variable
    cdef double t = ti
    
    # number of time steps
    cdef int n = int((te - ti) / delta_t) + 1
    # time step
    cdef int i = 0
    
    # https://stackoverflow.com/questions/25974975/cython-c-array-initialization
    # simulation time
    cdef double result_t[10001]
    result_t = [0.0] * 10001
    # state variable 
    cdef double result_x[10001]
    result_x = [0.0] * 10001
    
    # local variable for x'
    cdef double dx_dt = 0
    
    # initial values of time and state
    result_t[0] = ti
    result_x[0] = x0
    
    # time step loop
    for i in range(1, n):
        # slope
        dx_dt = diff_eq(result_x[i-1])
        # state variable of next time step
        result_x[i] = result_x[i-1] + dx_dt * delta_t
        # time step
        result_t[i] = result_t[i-1] + delta_t

    # result of simulation
    return result_t, result_x



* To compare, following cell prepares an equivalent simulation in python.

In [ ]:
def diff_eq_python(xi):

    # dx/dt + x = 0
    return -xi


def euler_python():

    # initial values
    ti = 0.0
    te = 10.0
    delta_t = 1e-3
    x0 = 1.0
    
    t = ti
    
    n = int((te - ti) / delta_t) + 1
    i = 0
    
    # prepare for buffers before the loop
    result_t = [0.0] * 10001
    result_x = [0.0] * 10001
    
    result_t[0] = ti
    result_x[0] = x0
    
    dx_dt = 0
    
    # time step loop
    for i in range(1, n):
        dx_dt = diff_eq_python(result_x[i-1])
        result_x[i] = result_x[i-1] + dx_dt * delta_t
        result_t[i] = result_t[i-1] + delta_t

    return result_t, result_x



In [ ]:
%%time
# measure time to calculate

t, x = euler_cython()



In [ ]:
%%time
# measure time to calculate

t_py, x_py = euler_python()



In [ ]:
import pylab as py


py.plot(t, x, 'o', label='cython')
py.plot(t_py, x_py, '-', label='python')
py.grid(True)
py.xlabel('t')
py.ylabel('y')
py.legend(loc=0)
py.show()



## Calling C/C++ functions

* Cython can call C/C++ functions as follows.
[[ref0](https://cython.readthedocs.io/en/latest/src/userguide/external_C_code.html)]
, [[ref1](https://stackoverflow.com/questions/37426534/how-can-i-import-an-external-c-function-into-an-ipython-notebook-using-cython)]
, [[ref2](https://stackoverflow.com/questions/19260253/cython-compiling-error-multiple-definition-of-functions)]
, [[ref3](https://media.readthedocs.org/pdf/cython/stable/cython.pdf)]
, [[ref4](http://www.scipy-lectures.org/advanced/interfacing_with_c/interfacing_with_c.html)]
 
 

### With `numpy` support

* If we need to use matrices and vectors frequently, combining `numpy` and cython may be helpful.

* Let's take a look at an example of calculating cosine values.

* Following is file `cos_cython_numpy.h`.

In [ ]:
%%writefile cos_cython_numpy.h
/*
    2.8.5.2. Numpy Support, 2.8.5. Cython, http://www.scipy-lectures.org/advanced/interfacing_with_c/interfacing_with_c.html#id13
*/
void cos_cython_numpy_c_func(double * in_array, double * out_array, int size);



* Following is file `cos_cython_numpy.c`.

In [ ]:
%%writefile cos_cython_numpy.c
/*
2.8.5.2. Numpy Support, 2.8.5. Cython, http://www.scipy-lectures.org/advanced/interfacing_with_c/interfacing_with_c.html#id13
*/

#include <math.h>

/*  Compute the cosine of each element in in_array, storing the result in
 *  out_array. */
void cos_cython_numpy_c_func(double * in_array, double * out_array, int size){
    int i;
    for(i=0;i<size;i++){
        out_array[i] = cos(in_array[i]);
    }
}



* Following is file `_cos_cython_numpy.pyx`.

In [ ]:
%%writefile _cos_cython_numpy.pyx
""" Example of wrapping a C function that takes C double arrays as input using
    the Numpy declarations from Cython 
    Valentin Haenel, 2.8.5.2. Numpy Support, 2.8.5. Cython, Scipy Lectures, Oct 18 2016, [Online] 
        Available: http://www.scipy-lectures.org/advanced/interfacing_with_c/interfacing_with_c.html#id13 
"""

""" Example of wrapping a C function that takes C double arrays as input using
    the Numpy declarations from Cython """

# cimport the Cython declarations for numpy
cimport numpy as np

# if you want to use the Numpy-C-API from Cython
# (not strictly necessary for this example, but good practice)
np.import_array()

# cdefine the signature of our c function
cdef extern from "cos_cython_numpy.h":
    void cos_cython_numpy_c_func (double * in_array, double * out_array, int size)

# create the wrapper code, with numpy type annotations
def cos_cython_numpy_py_func(np.ndarray[double, ndim=1, mode="c"] in_array not None,
                     np.ndarray[double, ndim=1, mode="c"] out_array not None):
    cos_cython_numpy_c_func(<double*> np.PyArray_DATA(in_array),
                <double*> np.PyArray_DATA(out_array),
                in_array.shape[0])



* Following is file `setup.py`.  Running this `.py` file would build the C function wrapper.

In [ ]:
%%writefile setup.py

# Valentin Haenel, 2.8. Interfacing with C,  Scipy Lectures, Oct 18 2016, [Online]
#   Available: http://www.scipy-lectures.org/advanced/interfacing_with_c/interfacing_with_c.html

from distutils.core import setup, Extension
# distutils : building and installing modules 
# https://docs.python.org/3/library/distutils.html

import numpy
from Cython.Distutils import build_ext

print('for NumPy Support of Cython '.ljust(60, '#'))
setup(cmdclass={'build_ext': build_ext},
      ext_modules=[Extension("cos_cython_numpy",
                             sources=['_cos_cython_numpy.pyx', "cos_cython_numpy.c"],
                             include_dirs=[numpy.get_include()])],
      )



In [ ]:
%%bash
python setup.py build_ext --inplace



* Now let's import the module with c wrapper function

In [ ]:
# the cython module including C function
import cos_cython_numpy

# to visualize the result
import pylab as py

# allocate arrays externally
x = py.arange(0, 2 * py.pi, 0.1)
y = py.empty_like(x)

# c wrapper function
cos_cython_numpy.cos_cython_numpy_py_func(x, y)

# plot the result
py.plot(x, py.cos(x), 'o', label='cos numpy')
py.plot(x, y, '.', label='cos cython')
py.show()



* Cleanup

In [ ]:
%%bash
rm cos_cython_numpy.h
rm cos_cython_numpy.c
rm _cos_cython_numpy.c
rm _cos_cython_numpy.pyx
rm setup.py



* Check results

In [ ]:
assert (y == py.cos(x)).all(), "Cython Result != Pylab Expected"



### Revisiting Euler's method

* Let's try wrapping a C/C++ version of the Euler's method above.

* Following is file `euler_cython_numpy.h`.

In [ ]:
%%writefile euler_cython_numpy.h
/*
    2.8.5.2. Numpy Support, 2.8.5. Cython, http://www.scipy-lectures.org/advanced/interfacing_with_c/interfacing_with_c.html#id13
*/
void euler_cython_c_function(const double * result_t, const double * result_x, const int size);



* Following is file `euler_cython_numpy.c`.

In [ ]:
%%writefile euler_cython_numpy.c
#include    <assert.h>


double diff_eq(double xi){
    return -xi;
}


void euler_cython_c_function(double * result_t, double * result_x, const int size){
    // Simulation start and end time
    double ti = 0.0;
    double te = 10.0;

    double delta_t = 1e-3;

    // Initial state
    double x0 = 1.0;
    double dx_dt = 0.0;
    
    int i=0;
    
    // Length of simulation
    const int n = (int) ((te - ti) / delta_t) + 1;

    // Check array size
    assert(size > n);

    // Set initial value    
    result_t[0] = ti;
    result_x[0] = x0;
    
    // Time step loop
    // Watch the last value of i here
    for (i=0; (n-1)>i; ++i){
        // Calculate derivative
        dx_dt = diff_eq(result_x[i]);
        // Calculate state value of the next step
        result_x[i+1] = result_x[i] + dx_dt * delta_t;
        // Calculate time of next step
        result_t[i+1] = result_t[i] + delta_t;
    }

    return;
}



* Following is file `_euler_cython_numpy.pyx`.

In [ ]:
%%writefile _euler_cython_numpy.pyx
# cimport the Cython declarations for numpy
cimport numpy as np

# if you want to use the Numpy-C-API from Cython
# (not strictly necessary for this example, but good practice)
np.import_array()

# cdefine the signature of our c function
cdef extern from "euler_cython_numpy.h":
    void euler_cython_c_function (double * result_t, double * result_x, int size)

# create the wrapper code, with numpy type annotations
def euler_cython_numpy_py_func(np.ndarray[double, ndim=1, mode="c"] in_array not None,
                     np.ndarray[double, ndim=1, mode="c"] out_array not None):
    euler_cython_c_function(<double*> np.PyArray_DATA(in_array),
                <double*> np.PyArray_DATA(out_array),
                in_array.shape[0])



* Following is file `setup.py`.

In [ ]:
%%writefile setup.py

# Valentin Haenel, 2.8. Interfacing with C,  Scipy Lectures, Oct 18 2016, [Online]
#   Available: http://www.scipy-lectures.org/advanced/interfacing_with_c/interfacing_with_c.html

from distutils.core import setup, Extension

import numpy
from Cython.Distutils import build_ext

print('for NumPy Support of Cython '.ljust(60, '#'))
setup(cmdclass={'build_ext': build_ext},
      ext_modules=[
                  Extension(
                        "euler_cython_numpy", 
                        sources=['_euler_cython_numpy.pyx', "euler_cython_numpy.c"], 
                        include_dirs=[numpy.get_include()]
                  ),
            ],
      )



In [ ]:
%%bash
python setup.py build_ext --inplace



* Let's import the cython c wrap module and run simulation.

In [ ]:
import pylab as py
import euler_cython_numpy

# As in cosine example, allocate the memory externally.
t_cy_wrap = py.zeros(10001)
x_cy_wrap = py.empty_like(t_cy_wrap)

# call the simulation function
euler_cython_numpy.euler_cython_numpy_py_func(t_cy_wrap, x_cy_wrap)

# plot result from python simulation
py.plot(t_py, x_py, 'o', label='python')

# plot result from cython c wrapper function
py.plot(t_cy_wrap, x_py, '.', label='cython wrap')

py.legend(loc=0)
py.show()



* Let's compare the calculation time again.

In [ ]:
%%time
# cython wrap with memory allocation

t_cy_wrap = py.zeros(10001)
x_cy_wrap = py.empty_like(t_cy_wrap)

euler_cython_numpy.euler_cython_numpy_py_func(t_cy_wrap, x_cy_wrap)



In [ ]:
t_cy_wrap = py.zeros(10001)
x_cy_wrap = py.empty_like(t_cy_wrap)



In [ ]:
%%time
# cython wrap without memory allocation

euler_cython_numpy.euler_cython_numpy_py_func(t_cy_wrap, x_cy_wrap)



In [ ]:
%%time
# cython

t, x = euler_cython()



In [ ]:
%%time
# python

t_py, x_py = euler_python()



* Clean up the files.

In [ ]:
%%bash
rm euler_cython_numpy.h
rm euler_cython_numpy.c
rm _euler_cython_numpy.c
rm _euler_cython_numpy.pyx
rm setup.py



* Check array type and dimension size

In [ ]:
assert isinstance(t_cy_wrap, py.ndarray), "t is not a numpy.ndarray"
assert isinstance(x_cy_wrap, py.ndarray), "x is not a numpy.ndarray"

# for now expect 1-dimensional arrays
assert 1 == t_cy_wrap.ndim, "t dimension > 1"
assert 1 == x_cy_wrap.ndim, "x dimension > 1"



* Check each element of the results

In [ ]:
for i in range(len(t_py)):
    assert t_cy_wrap[i] == t_py[i], f"t[{i}] Cython Result != Expected\n"
    assert x_cy_wrap[i] == x_py[i], f"x[{i}] Cython Result != Expected\n"

